### Chatgpt로 labeling하기

In [40]:
import pandas as pd

df = pd.read_csv('shinhan_duplicated_all.csv')
df[:1726]

,title
0,수신상품 기본금리 변경 안내
1,전산 시스템 개선 작업 안내
2,전산 시스템 작업 안내
3,영업점 통합 안내
4,오픈뱅킹 서비스 일부 제공기관 중단 안내
...,...
1721,대안정보 활용한 AI머신러닝 기반 신용평가모형 고도화 실시
1722,헝가리 대표사무소 개소
1723,"AI 기반의 새로운 고객소통 서비스 차별화! 신한은행, 자이냅스와 업무협약 체결"
1724,NaN


In [41]:
import re
import openai
import time
from tqdm import tqdm

# chatgpt API key 적어주기
openai.api_key = ''

# 모델 - GPT 3.5 Turbo 선택(기본)
model = "gpt-3.5-turbo"

label = [] # chatgpt로 뽑은 label -> list 형태
detail = [] # chatgpt가 내보낸 답변 그대로(list + 왜 그렇게 설정하였는지 설명)

for i in tqdm(range(len(df))):
    # 질문 작성하기
    query = f'''
            1. 소액 금융 상품 추천\n
            2. 안정 집중 금융 상품 추천\n
            3. 투자/주식 상품 추천\n
            4. 청년 특수 상품/혜택 추천\n
            5. 혜택 변경 소식\n
            6. 여행/숙박\n
            7. 문화/전시/예술\n
            8. 음식\n
            9. 교통\n
            10. 거주/주택/부동산\n
            \n
                {df['title'][i]} 와 같은 문장은 위 10개 label 중 어디에 속할지의 확률 값을 하나의 python list형태로 추론해줘\n
                list안의 원소의 합은 1이야.\n
            \n
            다음 형식을 사용해줘:\n
            list = 확률 값\n
            '''

    # 메시지 설정하기(변경X)
    messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
    ]

    # ChatGPT API 호출하기(변경X)
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages
    )

    # chatgpt 답변
    answer = response['choices'][0]['message']['content']
    print(i, df['title'][i])
    # print(answer)

    # detail
    detail_text = "'" + df['title'][i] + "'" + ' ' + answer
    detail.append(detail_text)

    # label
    tmp = re.split(r'\[|\]', detail_text)
    print(tmp)
    cnt = 0
    for i in range(len(tmp)):
        if tmp[i][0] == '0':
            label.append('[' + tmp[i] + ']')
            break
        elif i == len(tmp)-1:
            if cnt == len(tmp)-1:
                label.append(' ')
        else:
            cnt += 1

    # 무료 사용자는 1분에 3개의 질문을 호출해야 time out이 일어나지 않음
    time.sleep(21)


  0%|          | 0/277 [00:00<?, ?it/s]

1725 갤럭시코퍼레이션, 신한캐피탈과 전략적 업무협약 체결
["'갤럭시코퍼레이션, 신한캐피탈과 전략적 업무협약 체결' list = ", '0.2, 0.1, 0.3, 0.1, 0.05, 0.05, 0.05, 0.075, 0.075, 0.05', ' \n\n(1번 소액 금융 상품 추천은 20%, 2번 안정 집중 금융 상품 추천은 10%, 3번 투자/주식 상품 추천은 30%, 4번 청년 특수 상품/혜택 추천은 10%, 5번 혜택 변경 소식은 5%, 6번 여행/숙박은 5%, 7번 문화/전시/예술은 5%, 8번 음식은 7.5%, 9번 교통은 7.5%, 10번 거주/주택/부동산은 5%의 확률 값을 가집니다.)']


In [35]:
dfdf = pd.DataFrame({'title' : df, 'label' : label, 'detail' : detail})
dfdf

,title,label,detail
2751,첫 돌 맞이한 써니뱅크! 써니뱅크 1주년 기념식 개최,"[0.1, 0.1, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,...","'첫 돌 맞이한 써니뱅크! 써니뱅크 1주년 기념식 개최' list = [0.1, 0..."
2752,"신한銀, 금융권 최초 노란우산공제 모바일 신규 시행","[0.2, 0.1, 0.3, 0.1, 0.1, 0.05, 0.05, 0.05, 0....","'신한銀, 금융권 최초 노란우산공제 모바일 신규 시행' list = [0.2, 0...."
2753,"신한銀, 바젤III 적격 외화 후순위채 5억불 청약 성공","[0.1, 0.2, 0.15, 0.05, 0.05, 0.1, 0.1, 0.1, 0....","'신한銀, 바젤III 적격 외화 후순위채 5억불 청약 성공' list = [0.1,..."
2754,"신한銀, 고병원성 조류인플루엔자(AI) 피해기업 금융지원 실시","[0, 0.2, 0, 0.3, 0, 0, 0, 0, 0, 0.5]","'신한銀, 고병원성 조류인플루엔자(AI) 피해기업 금융지원 실시' list = [0..."
2755,"신한 써니뱅크, 금융권 최초 여권으로 비대면 계좌개설","[0.2, 0.1, 0.05, 0.15, 0.1, 0.1, 0.05, 0.1, 0....","'신한 써니뱅크, 금융권 최초 여권으로 비대면 계좌개설' list = [0.2, 0..."
...,...,...,...
2996,"신한銀, ‘신한 InsideBank’ 업그레이드 오픈","[0, 0.3, 0, 0.2, 0.1, 0.1, 0, 0, 0, 0.3]","'신한銀, ‘신한 InsideBank’ 업그레이드 오픈' list = [0, 0.3..."
2997,"신한銀, 전통시장 활성화 앞장서 소상공인시장진흥공단과 업무협약 체결","[0.2, 0.1, 0.1, 0.05, 0.05, 0.1, 0.05, 0.1, 0....","'신한銀, 전통시장 활성화 앞장서 소상공인시장진흥공단과 업무협약 체결' list =..."
2998,"신한銀, 아시아문화원 공식후원은행 협약 체결","[0.1, 0.1, 0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, ...","'신한銀, 아시아문화원 공식후원은행 협약 체결' list = [0.1, 0.1, 0..."
2999,"신한갤러리 2016년 첫 체험형 기획전 신한銀, ‘사람이랑 사랑해’展 개최!!","[0, 0, 0.2, 0, 0, 0.2, 0.1, 0.1, 0.1, 0.3]","'신한갤러리 2016년 첫 체험형 기획전 신한銀, ‘사람이랑 사랑해’展 개최!!' ..."


In [36]:
dfdf.to_csv('labeling_data.csv', index=False)